In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import os
import requests
import io
import zipfile

/tmp/ipykernel_56920/2038222792.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
table_names = {
  'MB_2021_AUST': 'Mesh Blocks - 2021',
  'SA1_2021_AUST': 'Statistical Areas Level 1 - 2021',
  'SA2_2021_AUST': 'Statistical Areas Level 2 - 2021',
  'SA3_2021_AUST': 'Statistical Areas Level 3 - 2021',
  'SA4_2021_AUST': 'Statistical Areas Level 4 - 2021',
  'GCCSA_2021_AUST': 'Greater Capital City Statistical Areas - 2021',
  'STE_2021_AUST': 'States and Territories - 2021',
  'AUS_2021_AUST': 'Australia - 2021',
  'INDIGENOUS_STRUCTURE_ALLOCATION_2021': 'Indigenous Structure - 2021',
  'ILOC_2021_AUST': 'Indigenous Locations - 2021',
  'IARE_2021_AUST': 'Indigenous Areas - 2021',
  'IREG_2021_AUST': 'Indigenous Regions - 2021',
  'LGA_2023_AUST': 'Local Government Areas - 2023',
  'LGA_2022_AUST': 'Local Government Areas - 2022',
  'LGA_2021_AUST': 'Local Government Areas - 2021',
  'SED_2022_AUST': 'State Electoral Divisions - 2022',
  'SED_2021_AUST': 'State Electoral Divisions - 2021',
  'CED_2021_AUST': 'Commonwealth Electoral Divisions - 2021',
  'POA_2021_AUST': 'Postal Areas - 2021',
  'TR_2021_AUST': 'Tourism Regions - 2021',
  'ADD_2021_AUST': 'Australian Drainage Divisions - 2021',
  'SAL_2021_AUST': 'Suburbs and Localities - 2021',
  'MB_DZN_2021_AUST': 'Destination Zones - 2021',
  'DZN_SA2_2021_AUST': 'Destination Zones to Statistical Areas Level 2 - 2021',
  'SUA_2021_AUST': 'Significant Urban Areas - 2021',
  'UCL_SOSR_SOS_2021_AUST': 'Urban Centres and Localities, Section of State and Section of State Range - 2021',
  'SUA_association_2016_2021': 'Significant Urban Area association - 2016 to 2021',
  'UCL_association_2016_2021': 'Urban Centre and Locality association - 2016 to 2021',
  'RA_2021_AUST': 'Remoteness Areas - 2021'
}


In [21]:
dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}

/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,5,6,8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}
/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}
/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}
/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { 

In [24]:
for k, df in dfs.items():
    df.to_csv(f'../data/core/mb-alloc/{k}.csv', index=False)
    print(k)

DZN_SA2_2021_AUST
RA_2021_AUST
MB_2021_AUST
LGA_2022_AUST
AUS_2021_AUST
SA3_2021_AUST
SED_2022_AUST
GCCSA_2021_AUST
UCL_association_2016_2021
CED_2021_AUST
UCL_SOSR_SOS_2021_AUST
LGA_2023_AUST
STE_2021_AUST
INDIGENOUS_STRUCTURE_ALLOCATION_2021
ADD_2021_AUST
SA2_2021_AUST
MB_DZN_2021_AUST
SED_2021_AUST
POA_2021_AUST
LGA_2021_AUST
ILOC_2021_AUST
IARE_2021_AUST
TR_2021_AUST
SUA_association_2016_2021
SA1_2021_AUST
SA4_2021_AUST
SUA_2021_AUST
SAL_2021_AUST
IREG_2021_AUST


In [23]:
dfs['DZN_SA2_2021_AUST']

,DZN_CODE_2021,SA2_CODE_2021,SA2_NAME_2021,STATE_CODE_2021,STATE_NAME_2021,AUS_CODE_2021,AUS_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021
0,110078098,101021007,Braidwood,1,New South Wales,AUS,Australia,1539.8516,http://linked.data.gov.au/dataset/asgsed3/DZN/...
1,110078099,101021007,Braidwood,1,New South Wales,AUS,Australia,234.6163,http://linked.data.gov.au/dataset/asgsed3/DZN/...
2,110078100,101021007,Braidwood,1,New South Wales,AUS,Australia,1643.8846,http://linked.data.gov.au/dataset/asgsed3/DZN/...
3,110088084,101021008,Karabar,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/DZN/...
4,110090001,101021009,Queanbeyan,1,New South Wales,AUS,Australia,1.9582,http://linked.data.gov.au/dataset/asgsed3/DZN/...
...,...,...,...,...,...,...,...,...,...
9324,997999799,997979799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/DZN/...
9325,ZZZZZZZZZ,ZZZZZZZZZ,Outside Australia,Z,Outside Australia,ZZZ,Outside Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/DZN/...
9326,0&&&&&&&&,&&&&&&&&&,POW not stated,&,POW not stated,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/DZN/...
9327,0@@@@@@@@,@@@@@@@@@,POW not applicable,@,POW not applicable,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/DZN/...


In [18]:
{k: df.columns for k, df in dfs.items() }

{'DZN_SA2_2021_AUST': Index(['Unnamed: 0', 'DZN_CODE_2021', 'SA2_CODE_2021', 'SA2_NAME_2021',
        'STATE_CODE_2021', 'STATE_NAME_2021', 'AUS_CODE_2021', 'AUS_NAME_2021',
        'AREA_ALBERS_SQKM', 'ASGS_LOCI_URI_2021'],
       dtype='object'),
 'RA_2021_AUST': Index(['Unnamed: 0', 'SA1_CODE_2021', 'RA_CODE_2021', 'RA_NAME_2021',
        'STATE_CODE_2021', 'STATE_NAME_2021', 'AUS_CODE_2021', 'AUS_NAME_2021',
        'AREA_ALBERS_SQKM', 'ASGS_LOCI_URI_2021'],
       dtype='object'),
 'MB_2021_AUST': Index(['Unnamed: 0', 'MB_CODE_2021', 'MB_CATEGORY_2021', 'CHANGE_FLAG_2021',
        'CHANGE_LABEL_2021', 'SA1_CODE_2021', 'SA2_CODE_2021', 'SA2_NAME_2021',
        'SA3_CODE_2021', 'SA3_NAME_2021', 'SA4_CODE_2021', 'SA4_NAME_2021',
        'GCCSA_CODE_2021', 'GCCSA_NAME_2021', 'STATE_CODE_2021',
        'STATE_NAME_2021', 'AUS_CODE_2021', 'AUS_NAME_2021', 'AREA_ALBERS_SQKM',
        'ASGS_LOCI_URI_2021'],
       dtype='object'),
 'LGA_2022_AUST': Index(['Unnamed: 0', 'MB_CODE_2021', 'LG